In [1]:
import gensim
import os
import collections
import smart_open
import random

In [2]:
from gensim import models,corpora,similarities

In [3]:
# Establecer nombres de archivo para trenes y datos de prueba

test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = 'datos.csv'
lee_test_file = './textos/S0004-06142006000500002-4.txt'

In [4]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [5]:

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
train_corpus[:2]

[TaggedDocument(words=['titulo', 'descripcion'], tags=[0]),
 TaggedDocument(words=['novedades', 'en', 'cirugã', 'de', 'columna', 'vertebral', 'en', 'en', 'este', 'artã', 'culo', 'de', 'revisiã³n', 'queremos', 'destacar', 'las', 'principales', 'novedades', 'curiosidades', 'alrededor', 'de', 'la', 'cirugã', 'de', 'columna', 'que', 'se', 'han', 'publicado', 'el', 'aã', 'aunque', 'el', 'aã', 'pasado', 'no', 'han', 'sido', 'publicadas', 'grandes', 'novedades', 'en', 'nuestra', 'rea', 'de', 'estudio', 'nosotros', 'queremos', 'hacer', 'hincapiã', 'en', 'los', 'aspectos', 'que', 'nos', 'han', 'parecido', 'mã', 'relevantes', 'sobre', 'cirugã', 'de', 'columna', 'dado', 'que', 'la', 'cirugã', 'de', 'columna', 'representa', 'una', 'importante', 'carga', 'asistencial', 'sanitaria', 'social', 'los', 'estudios', 'aquã', 'reflejados', 'intentan', 'dar', 'respuestas', 'las', 'mãºltiples', 'demandas', 'pendientes', 'asã', 'es', 'como', 'abordaremos', 'temas', 'como', 'la', 'artrodesis', 'cervical', 'vs'

In [7]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=55)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
# model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])

Wall time: 5.49 s


In [13]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    # inferir nuevos vectores para cada documento del corpus de entrenamiento
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    # Cacular simialidad del vector inferido con documentos en el modelo sims es una matriz que contiene el resultado organizado en orden descendente por ejemplo [(0, 0.9274195432662964), (48, 0.8267428278923035)] donde El elemento de tupla es doc_id y el segundo es el puntaje de simialidad
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
  #devuelve el rango del documento actual de la lista de documentos más similares
#asumimos que el documento más similar es el mismo y, por lo tanto, el rango debe estar en el índice
    rank = [docid for docid, sim in sims].index(doc_id) 
    ranks.append(rank)
    second_ranks.append(sims[1])

In [14]:
# cuente cómo se clasifica cada documento con respecto al corpus de capacitación
# la comprobación de un vector inferido contra un vector de entrenamiento es una especie de "comprobación de cordura"
# en cuanto a si el modelo se comporta de una manera útil y consistente, aunque no es un valor real de "precisión".
collections.Counter(ranks)

Counter({343: 1, 0: 524, 1: 1})

In [15]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (525): «paciente con sdt tras prostatectomã radical el sã ndrome de dã ficit de testosterona sdt define un cuadro clã nico bioquã mico asociado la edad que produce un detrimento en la calidad de vida de estos pacientes estudios recientes abogan por la seguridad del tratamiento hormonal con testosterona en pacientes con historia de cã ncer de prã³stata cp al no haberse demostrado inducciã³n de este tumor un total de cinco publicaciones presentan resultados de pacientes tratados con testosterona con historia de cp intervenidos mediante prostatectomã radical pr solo un paciente presentã³ recidiva bioquã mica durante el tratamiento sustitutivo la terapia de reemplazamiento con testosterona ha de ser indicada en pacientes seleccionados que se encuentren sintomã ticos con historia de cã ncer localizado de bajo riesgo tratado de manera satisfactoria buen control oncolã³gico las cifras de testosterona alcanzar mantener serã las mã nimas eficaces que permitan obtener una respuesta sint